In [285]:
from base64 import decode
import random 
import numpy as np
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

In [286]:
N = 20
POPULATION_SIZE = 50
OFFSPRING_SIZE = 30

In [287]:
#We discard duplicate elements
def problem(N, seed=42):
    """Generates the problem, also makes all blocks generated unique"""
    random.seed(seed)
    blocks_not_unique = [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]
    blocks_unique = np.unique(np.array(blocks_not_unique, dtype=object))
    return blocks_unique.tolist()

initial_list = problem(N)

the_list = list()
the_list.append((None, None, "initial"))
the_list_counter = 0
the_list_current_option = "initial"

In [288]:
def fitness(individual):
    return sum([len(l[0]) for l in zip(initial_list, individual) if l[1] is True])

def checkFeasibility(individual):
    covered = set()
    for p in zip(initial_list, individual):
        covered.update([n for n in p[0] if p[1]==True])
    return len(covered) == N

def select_parent(population, tournament_size = 2):
    subset = random.choices(population, k = tournament_size)
    return min(subset, key=lambda i: i [0])

def mutation(g):
    point = random.randint(0,len(g)-1)
    return g[:point] + [not g[point]] + g[point+1:]

def cross_over(g1, g2):
    cut = random.randint(0,len(g1))
    return g1[:cut] + g2[cut:]

def calculateMutationProbability(best_candidate):
    distance = abs(N - best_candidate[0])
    return 1-(distance/N)

def calculateMutationProbabilityDet2(best_candidate):
    global the_list, the_list_current_option

    probability_selected = 0.5
    probability_reason = ""

    # check if best changed (based on fitness func)
    if not best_candidate[0] == the_list[-1][0]:
        the_list = list()
        the_list.append(best_candidate)
    else:
        the_list.append(best_candidate)

    # if list is bigger than 10 select opositive of current best
    if len(the_list) > 10:

        if len(the_list) < 21:
            if best_candidate[2] == "mutation":
                probability_reason= "cross"
                probability_selected = 0.1
            else:
                probability_reason= "mutation"
                probability_selected = 0.9
        else:
            probability_reason = the_list_current_option

        if len(the_list) % 20 == 0:
            if the_list_current_option == "mutation":
                probability_reason= "cross"
                probability_selected = 0.1
            else:
                probability_reason= "mutation"
                probability_selected = 0.9
    else:
        probability_reason = "normal"
        probability_selected = calculateMutationProbability(best_candidate)

    the_list_current_option = probability_reason
    # print(f"{the_list_current_option} selected")
    return probability_selected

In [289]:
population = list()
while len(population) != POPULATION_SIZE:
    random_choices = random.choices([True, False], k=len(initial_list))
    fit_ind = (fitness(random_choices), random_choices)
    if checkFeasibility(random_choices) and fit_ind not in population:
        population.append(fit_ind)

In [ ]:
PARAMETERS = {
    "N":[5, 20, 100],
    "POPULATION_SIZE":[10,30,50,100],
    "OFFSPRING_SIZE":[20, 60, 100, 200]
}

configurations = {"configurations": []}
my_configs = ParameterGrid(PARAMETERS)
for config in my_configs:
    configurations["configurations"].append(config)

In [290]:
header="N,POPULATION_SIZE,OFFSPRING_SIZE,fitness \n"
with open("results.csv", "w") as csvf_header:
    csvf_header.write(header)

with open("results.csv", "a") as csvf:
    for idx in tqdm(range(len(configurations["configurations"]))):

        config = configurations["configurations"][idx]

        for __ in range(1000):
            offspring_pool = list()
            while len(offspring_pool) != OFFSPRING_SIZE:
                mutation_probability = calculateMutationProbabilityDet2(population[0], 5)
                if random.random() < calculateMutationProbability(population[0]):
                    p = select_parent(population)
                    new_individual = mutation(p[1])
                else:
                    while p1 == p2:
                        p1, p2 = select_parent(population), select_parent(population)
                    new_individual = cross_over(p1[1], p2[1])
                fit_off = (fitness(new_individual), new_individual)
                if checkFeasibility(new_individual) and fit_off not in offspring_pool:
                    offspring_pool.append(
                        (fitness(new_individual), new_individual))
            for o in offspring_pool:
                if o not in population:
                    population.append(o)
            population = sorted(population, key=lambda x: x[0])
            population = population[:POPULATION_SIZE]

        csvf.write(f"{config['N']},{config['POPULATION_SIZE'],{fitness}\n")



# history = ("mutation", fitness)

# if history[0] == "mutation": #se mutation (o crossover) è uguale per tutti
#     mutation_probability = 0.1

KeyboardInterrupt: 